In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import random
import re
import pandas as pd 

In [2]:
# WebDriver 설정
options = webdriver.ChromeOptions()
options.add_argument("window-size=1920x1080")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 스크롤 로직
def scroll_to_bottom(scrollable_element):
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
    while True:
        driver.execute_script("arguments[0].scrollTop += 600;", scrollable_element)
        time.sleep(1)
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)
        if new_height == last_height:
            break
        last_height = new_height

# 네이버 지도 접속
driver.get("https://map.naver.com/")
time.sleep(3)

# 검색어 입력 및 검색
search_box = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "input_search"))
)
search_box.send_keys("서초구 맛집")
search_box.send_keys(Keys.ENTER)
time.sleep(5)

# iframe 전환
try:
    iframe = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe"))
    )
    driver.switch_to.frame(iframe)
except Exception as e:
    print(f"iframe 전환 오류: {e}")
    driver.quit()
    exit()

# 결과 리스트 초기화
results = []
visited_urls = set()
page_number = 1
MAX_PAGES = 5  # 최대 페이지 수
MAX_RESULTS = 150  # 최대 결과 수 제한

while page_number <= MAX_PAGES and len(results) < MAX_RESULTS:
    try:
        # 스크롤을 맨 아래까지 내리기
        scrollable_element = driver.find_element(By.CLASS_NAME, "Ryr1F")
        scroll_to_bottom(scrollable_element)

        # 현재 페이지의 가게 리스트 가져오기
        store_elements = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CLASS_NAME, "place_bluelink"))
        )
        print(f"현재 {page_number}페이지) 총 {len(store_elements)}개의 가게를 찾았습니다.")

        for store in store_elements:
            if len(results) >= MAX_RESULTS:
                print("최대 결과 수에 도달하여 크롤링을 종료합니다.")
                break

            try:
                store.click()
                time.sleep(random.uniform(3, 5))

                # 상세 iframe 전환
                driver.switch_to.default_content()
                detail_iframe = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#entryIframe"))
                )
                driver.switch_to.frame(detail_iframe)

                # 페이지 소스 파싱
                html = driver.page_source
                soup = BeautifulSoup(html, "lxml")

                # 가게 정보 추출
                place_name = soup.select_one("span.GHAhO").text.strip() if soup.select_one("span.GHAhO") else "N/A"
                category = soup.select_one("span.lnJFt").text.strip() if soup.select_one("span.lnJFt") else "N/A"
                address = soup.select_one("span.LDgIH").text.strip() if soup.select_one("span.LDgIH") else "N/A"
                current_url = driver.current_url
                res_code = re.findall(r"place/(\d+)", current_url)
                store_url = f'https://pcmap.place.naver.com/restaurant/{res_code[0]}/review/visitor#' if res_code else "N/A"

                # 중복 여부 확인
                if store_url not in visited_urls:
                    visited_urls.add(store_url)
                    results.append({
                        "가게 이름": place_name,
                        "업종구분": category,
                        "주소": address,
                        "URL": store_url
                    })
                    print(f"{len(results)}. {place_name}")

                # iframe 복귀
                driver.switch_to.default_content()
                iframe = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe"))
                )
                driver.switch_to.frame(iframe)

            except Exception as e:
                print(f"가게 크롤링 중 오류 발생: {e}")

        if len(results) >= MAX_RESULTS:
            print("최대 결과 수에 도달하여 크롤링을 종료합니다.")
            break

        # '다음' 버튼 클릭
        if page_number < MAX_PAGES:
            try:
                next_button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, '//a[@class="eUTV2" and @aria-disabled="false"]'))
                )
                driver.execute_script("arguments[0].click();", next_button)
                print(f"'다음' 버튼 클릭: 페이지 {page_number + 1}로 이동 중")
                time.sleep(random.uniform(5, 10))

                # iframe 전환
                driver.switch_to.default_content()
                iframe = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "iframe#searchIframe"))
                )
                driver.switch_to.frame(iframe)

            except Exception as e:
                print(f"'다음' 버튼 클릭 중 오류 발생: {e}")
                break

        # 페이지 번호 증가
        page_number += 1

    except Exception as e:
        print(f"페이지 크롤링 중 오류 발생: {e}")
        break

# CSV 파일 저장
file_name = "Seocho-gu_restaurants.csv"
df = pd.DataFrame(results)
df.to_csv(file_name, index=False, encoding="utf-8-sig")

print(f"크롤링 완료. 결과는 {file_name}에 저장되었습니다.")
driver.quit()

현재 1페이지) 총 76개의 가게를 찾았습니다.
1. 강남한우정육식당
2. 큐브뮤직타운
3. 이자카야나무 서초점
4. 자매수산 강남본점
5. 사보텐 신논현점
6. 디오디아2.0
7. 낙원타코 강남역점
8. 바다당
9. 텍사스데브라질 센트럴시티점
10. 오감쭈꾸미
11. 미담재 양재역점
12. 플랫나인
13. 린참치
14. 호왕
15. JW 메리어트 호텔 서울 타마유라
16. 갈비사랑 양재본점
17. 리에또
18. 도우룸
19. 하이디라오 서초지점
20. 로리스 더 프라임 립
21. 보글이 생태탕왕코다리 양재역점
22. 이가네양꼬치 양재2호직영점
23. 석문어 양재점
24. 디엠룸술집 강남점
25. 정성해장국 교대본점
26. 온센 강남점
27. 차알 파미에스테이션점
28. 이자카야나무 사당점
29. 감격시대
30. 고깃집열
31. 육품
32. 은행골 강남역점
33. 교대이층집 교대본점
34. 미담재 교대역점
35. 정고기
36. JW 메리어트 호텔 서울 플레이버즈
37. 상석 논현본점
38. 케르반 파미에스테이션
39. 더 리버사이드 호텔 뷔페 더가든키친
40. 매우매오 파미에스테이션 강남점
41. 봉산옥 서초동본점
42. 이자카야 열
43. 갈비세상 양재본점
44. 놉스 서래마을점
45. 최고의 한우 명우
46. 신미경정통춘천닭갈비
47. 도셰프
48. 서울붕어베이커리
49. 다리원
50. 백화네부엌
51. 백년옥
52. 딘타이펑 강남점
53. 세광양대창 교대본점
54. 정담이 깃든 사랑채
55. 정담은
56. 무샤
57. 따롱이훠궈 강남점
58. 토박이별당
59. 탐라도야지
60. 양재오제섞어 양재본점
61. 7번방 강남점
62. 샤브지오
63. 미나미 서초점
64. 장수해물대첩 방배본점
65. 세광양대창 양재점
66. 김태완스시 방배본점
67. 무월식탁 파미에스테이션점
68. 고메램 교대점
69. 올리브에비뉴
70. 솔솥 교대역점
71. 비지엠커피
72. 족슐랭 양재시민의숲역점
73. 루엘드파리
74. 설마중
75. 담뿍화로된장찌개 교대점
76. 강남77


가게 크롤링 중 오류 발생: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=131.0.6778.86); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x002C3433+25059]
	(No symbol) [0x0024CE34]
	(No symbol) [0x0012BEC3]
	(No symbol) [0x001319D1]
	(No symbol) [0x001338EA]
	(No symbol) [0x00133967]
	(No symbol) [0x00171832]
	(No symbol) [0x00165EFF]
	(No symbol) [0x00191EFC]
	(No symbol) [0x00165A24]
	(No symbol) [0x00192194]
	(No symbol) [0x001AB51E]
	(No symbol) [0x00191C96]
	(No symbol) [0x00163FAC]
	(No symbol) [0x00164F3D]
	GetHandleVerifier [0x005B5593+3113795]
	GetHandleVerifier [0x005CA25A+3198986]
	GetHandleVerifier [0x005C2A32+3168226]
	GetHandleVerifier [0x003632A0+680016]
	(No symbol) [0x0025577D]
	(No symbol) [0x00252A28]
	(No symbol) [0x00252BC5]
	(No symbol) [0x00245820]
	BaseThreadInitThunk [0x775BFCC9+25

'다음' 버튼 클릭: 페이지 4로 이동 중
현재 4페이지) 총 70개의 가게를 찾았습니다.
131. 분코다이닝 사당점
132. 식껍 서초직영점
133. 고깃집열 논현역점
134. 돌짱
135. 채미가 서초교대점
136. 비노인빌라 Vino in Villa
137. 조개창고 방배점
138. 프릳츠 양재점
139. 진구곱창
140. 대성도가니탕 서초점
141. 옛날농장 매헌점
142. 라멘모토
143. 미우야
144. 바베큐랩
145. 흑산도홍탁
146. 마루심 반포점
147. 대나무골한정식
148. 라그릴리아 양재점
149. 일식대서양 서초교대점
150. 한성양꼬치 교대2호점
최대 결과 수에 도달하여 크롤링을 종료합니다.
최대 결과 수에 도달하여 크롤링을 종료합니다.
크롤링 완료. 결과는 Seocho-gu_restaurants.csv에 저장되었습니다.
